- Starting point: https://quantdev.ssri.psu.edu/sites/qdev/files/topic_modeling_tutorial-Gutenberg-chapter_as_document_0.html

In [64]:
install.packages("rlist")

also installing the dependencies ‘XML’, ‘data.table’




The downloaded binary packages are in
	/var/folders/6z/gpf2xmrs09b984lldpk58mg0vxysrb/T//RtmpgmjZ2o/downloaded_packages


In [1]:
library(tm) #to process text
library(topicmodels)
library(dplyr)
library(tidytext)
library(tidyverse)
library(SnowballC) # for stemming
library(stringr)
library(ldatuning)
library(gutenbergr)

#library(rlist) 

#append is functionally identical to list.append for what I want to do
#rlist's list.append() does allow you to avoid variable reassignment, though

Loading required package: NLP

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ readr   1.1.1
✔ tibble  1.4.2     ✔ purrr   0.2.4
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ ggplot2 2.2.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ ggplot2::annotate() masks NLP::annotate()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()


In [2]:
gutenberg_works(str_detect(author, "Wells, H. G."))

gutenberg_id,title,author,gutenberg_author_id,language,gutenberg_bookshelf,rights,has_text
35,The Time Machine,"Wells, H. G. (Herbert George)",30,en,Science Fiction/Movie Books,Public domain in the USA.,TRUE
36,The War of the Worlds,"Wells, H. G. (Herbert George)",30,en,Movie Books/Science Fiction,Public domain in the USA.,TRUE
159,The Island of Doctor Moreau,"Wells, H. G. (Herbert George)",30,en,Best Books Ever Listings/Science Fiction/Movie Books,Public domain in the USA.,TRUE
456,"The Door in the Wall, and Other Stories","Wells, H. G. (Herbert George)",30,en,Science Fiction,Public domain in the USA.,TRUE
524,Ann Veronica: A Modern Love Story,"Wells, H. G. (Herbert George)",30,en,NA,Public domain in the USA.,TRUE
718,Tono-Bungay,"Wells, H. G. (Herbert George)",30,en,Best Books Ever Listings,Public domain in the USA.,TRUE
775,When the Sleeper Wakes,"Wells, H. G. (Herbert George)",30,en,Science Fiction,Public domain in the USA.,TRUE
780,The War in the Air,"Wells, H. G. (Herbert George)",30,en,NA,Public domain in the USA.,TRUE
1013,The First Men in the Moon,"Wells, H. G. (Herbert George)",30,en,Science Fiction/Movie Books,Public domain in the USA.,TRUE
1046,"God, the Invisible King","Wells, H. G. (Herbert George)",30,en,"Bestsellers, American, 1895-1923",Public domain in the USA.,TRUE


In [60]:
gutenberg_works(str_detect(gutenberg_bookshelf, "Science Fiction"))

gutenberg_id,title,author,gutenberg_author_id,language,gutenberg_bookshelf,rights,has_text
35,The Time Machine,"Wells, H. G. (Herbert George)",30,en,Science Fiction/Movie Books,Public domain in the USA.,TRUE
36,The War of the Worlds,"Wells, H. G. (Herbert George)",30,en,Movie Books/Science Fiction,Public domain in the USA.,TRUE
42,The Strange Case of Dr. Jekyll and Mr. Hyde,"Stevenson, Robert Louis",35,en,Horror/Movie Books/Precursors of Science Fiction/Gothic Fiction,Public domain in the USA.,TRUE
62,A Princess of Mars,"Burroughs, Edgar Rice",48,en,Best Books Ever Listings/Science Fiction,Public domain in the USA.,TRUE
64,The Gods of Mars,"Burroughs, Edgar Rice",48,en,Science Fiction,Public domain in the USA.,TRUE
68,Warlord of Mars,"Burroughs, Edgar Rice",48,en,Science Fiction,Public domain in the USA.,TRUE
72,"Thuvia, Maid of Mars","Burroughs, Edgar Rice",48,en,Science Fiction,Public domain in the USA.,TRUE
83,"From the Earth to the Moon; and, Round the Moon","Verne, Jules",60,en,Science Fiction/Movie Books,Public domain in the USA.,TRUE
84,"Frankenstein; Or, The Modern Prometheus","Shelley, Mary Wollstonecraft",61,en,Gothic Fiction/Movie Books/Precursors of Science Fiction/Science Fiction by Women,Public domain in the USA.,TRUE
86,A Connecticut Yankee in King Arthur's Court,"Twain, Mark",53,en,Precursors of Science Fiction/Arthurian Legends,Public domain in the USA.,TRUE


In [3]:
gutenberg_ids<-gutenberg_works(str_detect(author, "Wells, H. G."))$gutenberg_id
head(gutenberg_ids)

[1]  35  36 159 456 524 718

In [4]:
gutenberg_ids <- append(gutenberg_ids,
       gutenberg_works(
            str_detect(
                gutenberg_bookshelf, "Science Fiction"))$gutenberg_id)
gutenberg_ids <- append(gutenberg_ids,
       gutenberg_works(
            str_detect(
                gutenberg_bookshelf, "Fantasy"))$gutenberg_id)
gutenberg_ids <- append(gutenberg_ids,
       gutenberg_works(
            str_detect(
                gutenberg_bookshelf, "Adventure"))$gutenberg_id)
gutenberg_ids <- append(gutenberg_ids,
       gutenberg_works(
            str_detect(
                gutenberg_bookshelf, "Horror"))$gutenberg_id)
gutenberg_ids <- append(gutenberg_ids,
       gutenberg_works(
            str_detect(
                gutenberg_bookshelf, "Children's Literature"))$gutenberg_id)

In [5]:
length(gutenberg_ids)

[1] 1803

In [6]:
length(unique(gutenberg_ids))

[1] 1761

In [7]:
gutenberg_ids<-unique(gutenberg_ids)

In [8]:
books <- gutenberg_download(c(gutenberg_ids), meta_fields = c("title","author"))
# gutenberg_download(c(gutenberg_ids), meta_fields = "title")

Determining mirror for Project Gutenberg from http://www.gutenberg.org/robot/harvest
Using mirror http://aleph.gutenberg.org
Warning message in .f(.x[[i]], ...):
“Could not download a book at http://aleph.gutenberg.org/1/9/5/1/19513/19513.zip”Multiple files in zip: reading '25438.txt'
Warning message in .f(.x[[i]], ...):
“Could not download a book at http://aleph.gutenberg.org/1/9/4/6/19467/19467.zip”Warning message in .f(.x[[i]], ...):
“Could not download a book at http://aleph.gutenberg.org/1/9/6/7/19678/19678.zip”Multiple files in zip: reading '11757.txt'
Warning message in .f(.x[[i]], ...):
“Could not download a book at http://aleph.gutenberg.org/1/9/5/1/19514/19514.zip”Warning message in .f(.x[[i]], ...):
“Could not download a book at http://aleph.gutenberg.org/1/9/6/3/19637/19637.zip”

### Saving the dataframe in a few formats:

In [9]:
save(books,file="books.Rda")

In [17]:
# # Install feather
# devtools::install_github("wesm/feather/R")

# library(feather)

from URL https://api.github.com/repos/wesm/feather/zipball/master
Installing feather
'/Library/Frameworks/R.framework/Resources/bin/R' --no-site-file --no-environ  \
  --no-save --no-restore --quiet CMD INSTALL  \
  '/private/var/folders/6z/gpf2xmrs09b984lldpk58mg0vxysrb/T/Rtmp1MMCK3/devtools7366d321f4e/wesm-feather-be5141b/R'  \
  --library='/Library/Frameworks/R.framework/Versions/3.5/Resources/library'  \
  --install-tests 



In [19]:
# path <- "books.feather"
# write_feather(books, path)

In [ ]:
# # Encoding is latin1
# write.csv(books,file="books.csv",row.names=FALSE)

# # Alternatively, this should work if I really want it in UTF-8
# text2 <- iconv(books, from = "latin1", to = "UTF-8")
# write.csv(text2,file="books.txt",row.names=FALSE)

In [11]:
head(books)

gutenberg_id,text,title,author
11,ALICE'S ADVENTURES IN WONDERLAND,Alice's Adventures in Wonderland,"Carroll, Lewis"
11,,Alice's Adventures in Wonderland,"Carroll, Lewis"
11,Lewis Carroll,Alice's Adventures in Wonderland,"Carroll, Lewis"
11,,Alice's Adventures in Wonderland,"Carroll, Lewis"
11,THE MILLENNIUM FULCRUM EDITION 3.0,Alice's Adventures in Wonderland,"Carroll, Lewis"
11,,Alice's Adventures in Wonderland,"Carroll, Lewis"
